# Raspagem das Entradas da Pokédex do Site Bulbapédia

Neste notebook irei fazer um scraping das descrições dos 898 pokémons até a VIII geração do site https://bulbagarden.net/ para posterior análise por NLP.

**Importação de bibliotecas**

In [1]:
import bs4
import urllib.request as urllib_request
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

**Dividindo os 898 pokémons em listas de 100 cada**

In [2]:
pkmn1 = ["Bulbasaur", "Ivysaur", "Venusaur", "Charmander", "Charmeleon", "Charizard", "Squirtle", "Wartortle", "Blastoise", "Caterpie", 
         "Metapod", "Butterfree", "Weedle", "Kakuna", "Beedrill", "Pidgey", "Pidgeotto", "Pidgeot", "Rattata", "Raticate", "Spearow", 
         "Fearow", "Ekans", "Arbok", "Pikachu", "Raichu", "Sandshrew", "Sandslash", "Nidoran%E2%99%80", "Nidorina", "Nidoqueen", "Nidoran%E2%99%82", 
         "Nidorino", "Nidoking", "Clefairy", "Clefable", "Vulpix", "Ninetales", "Jigglypuff", "Wigglytuff", "Zubat", "Golbat", 
         "Oddish", "Gloom", "Vileplume", "Paras", "Parasect", "Venonat", "Venomoth", "Diglett", "Dugtrio", "Meowth", "Persian", 
         "Psyduck", "Golduck", "Mankey", "Primeape", "Growlithe", "Arcanine", "Poliwag", "Poliwhirl", "Poliwrath", "Abra", "Kadabra", 
         "Alakazam", "Machop", "Machoke", "Machamp", "Bellsprout", "Weepinbell", "Victreebel", "Tentacool", "Tentacruel", "Geodude", 
         "Graveler", "Golem", "Ponyta", "Rapidash", "Slowpoke", "Slowbro", "Magnemite", "Magneton", "Farfetch%27d", "Doduo", "Dodrio", 
         "Seel", "Dewgong", "Grimer", "Muk", "Shellder", "Cloyster", "Gastly", "Haunter", "Gengar", "Onix", "Drowzee", "Hypno", 
         "Krabby", "Kingler", "Voltorb"]
pkmn2 = ["Electrode", "Exeggcute", "Exeggutor", "Cubone", "Marowak", "Hitmonlee", "Hitmonchan", "Lickitung", "Koffing", "Weezing", 
         "Rhyhorn", "Rhydon", "Chansey", "Tangela", "Kangaskhan", "Horsea", "Seadra", "Goldeen", "Seaking", "Staryu", "Starmie", 
         "Mr._Mime", "Scyther", "Jynx", "Electabuzz", "Magmar", "Pinsir", "Tauros", "Magikarp", "Gyarados", "Lapras", "Ditto", 
         "Eevee", "Vaporeon", "Jolteon", "Flareon", "Porygon", "Omanyte", "Omastar", "Kabuto", "Kabutops", "Aerodactyl", "Snorlax", 
         "Articuno", "Zapdos", "Moltres", "Dratini", "Dragonair", "Dragonite", "Mewtwo", "Mew", "Chikorita", "Bayleef", "Meganium", 
         "Cyndaquil", "Quilava", "Typhlosion", "Totodile", "Croconaw", "Feraligatr", "Sentret", "Furret", "Hoothoot", "Noctowl", 
         "Ledyba", "Ledian", "Spinarak", "Ariados", "Crobat", "Chinchou", "Lanturn", "Pichu", "Cleffa", "Igglybuff", "Togepi", 
         "Togetic", "Natu", "Xatu", "Mareep", "Flaaffy", "Ampharos", "Bellossom", "Marill", "Azumarill", "Sudowoodo", "Politoed", 
         "Hoppip", "Skiploom", "Jumpluff", "Aipom", "Sunkern", "Sunflora", "Yanma", "Wooper", "Quagsire", "Espeon", "Umbreon", 
         "Murkrow", "Slowking", "Misdreavus"]
pkmn3 = ["Unown", "Wobbuffet", "Girafarig", "Pineco", "Forretress", "Dunsparce", "Gligar", "Steelix", "Snubbull", "Granbull", 
         "Qwilfish", "Scizor", "Shuckle", "Heracross", "Sneasel", "Teddiursa", "Ursaring", "Slugma", "Magcargo", "Swinub", 
         "Piloswine", "Corsola", "Remoraid", "Octillery", "Delibird", "Mantine", "Skarmory", "Houndour", "Houndoom", "Kingdra", 
         "Phanpy", "Donphan", "Porygon2", "Stantler", "Smeargle", "Tyrogue", "Hitmontop", "Smoochum", "Elekid", "Magby", 
         "Miltank", "Blissey", "Raikou", "Entei", "Suicune", "Larvitar", "Pupitar", "Tyranitar", "Lugia", "Ho-Oh", "Celebi", 
         "Treecko", "Grovyle", "Sceptile", "Torchic", "Combusken", "Blaziken", "Mudkip", "Marshtomp", "Swampert", "Poochyena", 
         "Mightyena", "Zigzagoon", "Linoone", "Wurmple", "Silcoon", "Beautifly", "Cascoon", "Dustox", "Lotad", "Lombre", "Ludicolo", 
         "Seedot", "Nuzleaf", "Shiftry", "Taillow", "Swellow", "Wingull", "Pelipper", "Ralts", "Kirlia", "Gardevoir", "Surskit", 
         "Masquerain", "Shroomish", "Breloom", "Slakoth", "Vigoroth", "Slaking", "Nincada", "Ninjask", "Shedinja", "Whismur", 
         "Loudred", "Exploud", "Makuhita", "Hariyama", "Azurill", "Nosepass"]
pkmn4 = ["Skitty", "Delcatty", "Sableye", "Mawile", "Aron", "Lairon", "Aggron", "Meditite", "Medicham", "Electrike", "Manectric", 
         "Plusle", "Minun", "Volbeat", "Illumise", "Roselia", "Gulpin", "Swalot", "Carvanha", "Sharpedo", "Wailmer", "Wailord", 
         "Numel", "Camerupt", "Torkoal", "Spoink", "Grumpig", "Spinda", "Trapinch", "Vibrava", "Flygon", "Cacnea", "Cacturne", 
         "Swablu", "Altaria", "Zangoose", "Seviper", "Lunatone", "Solrock", "Barboach", "Whiscash", "Corphish", "Crawdaunt", 
         "Baltoy", "Claydol", "Lileep", "Cradily", "Anorith", "Armaldo", "Feebas", "Milotic", "Castform", "Kecleon", "Shuppet", 
         "Banette", "Duskull", "Dusclops", "Tropius", "Chimecho", "Absol", "Wynaut", "Snorunt", "Glalie", "Spheal", "Sealeo", 
         "Walrein", "Clamperl", "Huntail", "Gorebyss", "Relicanth", "Luvdisc", "Bagon", "Shelgon", "Salamence", "Beldum", "Metang", 
         "Metagross", "Regirock", "Regice", "Registeel", "Latias", "Latios", "Kyogre", "Groudon", "Rayquaza", "Jirachi", "Deoxys", 
         "Turtwig", "Grotle", "Torterra", "Chimchar", "Monferno", "Infernape", "Piplup", "Prinplup", "Empoleon", "Starly", "Staravia", 
         "Staraptor", "Bidoof"]
pkmn5 = ["Bibarel", "Kricketot", "Kricketune", "Shinx", "Luxio", "Luxray", "Budew", "Roserade", "Cranidos", "Rampardos", "Shieldon", 
         "Bastiodon", "Burmy", "Wormadam", "Mothim", "Combee", "Vespiquen", "Pachirisu", "Buizel", "Floatzel", "Cherubi", "Cherrim", 
         "Shellos", "Gastrodon", "Ambipom", "Drifloon", "Drifblim", "Buneary", "Lopunny", "Mismagius", "Honchkrow", "Glameow", 
         "Purugly", "Chingling", "Stunky", "Skuntank", "Bronzor", "Bronzong", "Bonsly", "Mime_Jr.", "Happiny", "Chatot", "Spiritomb", 
         "Gible", "Gabite", "Garchomp", "Munchlax", "Riolu", "Lucario", "Hippopotas", "Hippowdon", "Skorupi", "Drapion", "Croagunk", 
         "Toxicroak", "Carnivine", "Finneon", "Lumineon", "Mantyke", "Snover", "Abomasnow", "Weavile", "Magnezone", "Lickilicky", 
         "Rhyperior", "Tangrowth", "Electivire", "Magmortar", "Togekiss", "Yanmega", "Leafeon", "Glaceon", "Gliscor", "Mamoswine", 
         "Porygon-Z", "Gallade", "Probopass", "Dusknoir", "Froslass", "Rotom", "Uxie", "Mesprit", "Azelf", "Dialga", "Palkia", 
         "Heatran", "Regigigas", "Giratina", "Cresselia", "Phione", "Manaphy", "Darkrai", "Shaymin", "Arceus", "Victini", "Snivy", 
         "Servine", "Serperior", "Tepig", "Pignite"]
pkmn6 = ["Emboar", "Oshawott", "Dewott", "Samurott", "Patrat", "Watchog", "Lillipup", "Herdier", "Stoutland", "Purrloin", "Liepard", 
         "Pansage", "Simisage", "Pansear", "Simisear", "Panpour", "Simipour", "Munna", "Musharna", "Pidove", "Tranquill", "Unfezant", 
         "Blitzle", "Zebstrika", "Roggenrola", "Boldore", "Gigalith", "Woobat", "Swoobat", "Drilbur", "Excadrill", "Audino", "Timburr", 
         "Gurdurr", "Conkeldurr", "Tympole", "Palpitoad", "Seismitoad", "Throh", "Sawk", "Sewaddle", "Swadloon", "Leavanny", 
         "Venipede", "Whirlipede", "Scolipede", "Cottonee", "Whimsicott", "Petilil", "Lilligant", "Basculin", "Sandile", "Krokorok", 
         "Krookodile", "Darumaka", "Darmanitan", "Maractus", "Dwebble", "Crustle", "Scraggy", "Scrafty", "Sigilyph", "Yamask", 
         "Cofagrigus", "Tirtouga", "Carracosta", "Archen", "Archeops", "Trubbish", "Garbodor", "Zorua", "Zoroark", "Minccino", 
         "Cinccino", "Gothita", "Gothorita", "Gothitelle", "Solosis", "Duosion", "Reuniclus", "Ducklett", "Swanna", "Vanillite", 
         "Vanillish", "Vanilluxe", "Deerling", "Sawsbuck", "Emolga", "Karrablast", "Escavalier", "Foongus", "Amoonguss", "Frillish", 
         "Jellicent", "Alomomola", "Joltik", "Galvantula", "Ferroseed", "Ferrothorn", "Klink"]
pkmn7 = ["Klang", "Klinklang", "Tynamo", "Eelektrik", "Eelektross", "Elgyem", "Beheeyem", "Litwick", "Lampent", "Chandelure", "Axew", 
         "Fraxure", "Haxorus", "Cubchoo", "Beartic", "Cryogonal", "Shelmet", "Accelgor", "Stunfisk", "Mienfoo", "Mienshao", 
         "Druddigon", "Golett", "Golurk", "Pawniard", "Bisharp", "Bouffalant", "Rufflet", "Braviary", "Vullaby", "Mandibuzz", 
         "Heatmor", "Durant", "Deino", "Zweilous", "Hydreigon", "Larvesta", "Volcarona", "Cobalion", "Terrakion", "Virizion", 
         "Tornadus", "Thundurus", "Reshiram", "Zekrom", "Landorus", "Kyurem", "Keldeo", "Meloetta", "Genesect", "Chespin", 
         "Quilladin", "Chesnaught", "Fennekin", "Braixen", "Delphox", "Froakie", "Frogadier", "Greninja", "Bunnelby", "Diggersby", 
         "Fletchling", "Fletchinder", "Talonflame", "Scatterbug", "Spewpa", "Vivillon", "Litleo", "Pyroar", "Flab%C3%A9b%C3%A9", 
         "Floette", "Florges", "Skiddo", "Gogoat", "Pancham", "Pangoro", "Furfrou", "Espurr", "Meowstic", "Honedge", "Doublade", 
         "Aegislash", "Spritzee", "Aromatisse", "Swirlix", "Slurpuff", "Inkay", "Malamar", "Binacle", "Barbaracle", "Skrelp", 
         "Dragalge", "Clauncher", "Clawitzer", "Helioptile", "Heliolisk", "Tyrunt", "Tyrantrum", "Amaura", "Aurorus"]
pkmn8 = ["Sylveon", "Hawlucha", "Dedenne", "Carbink", "Goomy", "Sliggoo", "Goodra", "Klefki", "Phantump", "Trevenant", "Pumpkaboo", 
         "Gourgeist", "Bergmite", "Avalugg", "Noibat", "Noivern", "Xerneas", "Yveltal", "Zygarde", "Diancie", "Hoopa", "Volcanion", 
         "Rowlet", "Dartrix", "Decidueye", "Litten", "Torracat", "Incineroar", "Popplio", "Brionne", "Primarina", "Pikipek", 
         "Trumbeak", "Toucannon", "Yungoos", "Gumshoos", "Grubbin", "Charjabug", "Vikavolt", "Crabrawler", "Crabominable", 
         "Oricorio", "Cutiefly", "Ribombee", "Rockruff", "Lycanroc", "Wishiwashi", "Mareanie", "Toxapex", "Mudbray", "Mudsdale", 
         "Dewpider", "Araquanid", "Fomantis", "Lurantis", "Morelull", "Shiinotic", "Salandit", "Salazzle", "Stufful", "Bewear", 
         "Bounsweet", "Steenee", "Tsareena", "Comfey", "Oranguru", "Passimian", "Wimpod", "Golisopod", "Sandygast", "Palossand", 
         "Pyukumuku", "Type:_Null", "Silvally", "Minior", "Komala", "Turtonator", "Togedemaru", "Mimikyu", "Bruxish", "Drampa", 
         "Dhelmise", "Jangmo-o", "Hakamo-o", "Kommo-o", "Tapu_Koko", "Tapu_Lele", "Tapu_Bulu", "Tapu_Fini", "Cosmog", "Cosmoem", 
         "Solgaleo", "Lunala", "Nihilego", "Buzzwole", "Pheromosa", "Xurkitree", "Celesteela", "Kartana", "Guzzlord"]
pkmn9 = ["Necrozma", "Magearna", "Marshadow", "Poipole", "Naganadel", "Stakataka", "Blacephalon", "Zeraora", "Meltan", "Melmetal", 
         "Grookey", "Thwackey", "Rillaboom", "Scorbunny", "Raboot", "Cinderace", "Sobble", "Drizzile", "Inteleon", "Skwovet", 
         "Greedent", "Rookidee", "Corvisquire", "Corviknight", "Blipbug", "Dottler", "Orbeetle", "Nickit", "Thievul", "Gossifleur", 
         "Eldegoss", "Wooloo", "Dubwool", "Chewtle", "Drednaw", "Yamper", "Boltund", "Rolycoly", "Carkol", "Coalossal", "Applin", 
         "Flapple", "Appletun", "Silicobra", "Sandaconda", "Cramorant", "Arrokuda", "Barraskewda", "Toxel", "Toxtricity", 
         "Sizzlipede", "Centiskorch", "Clobbopus", "Grapploct", "Sinistea", "Polteageist", "Hatenna", "Hattrem", "Hatterene", 
         "Impidimp", "Morgrem", "Grimmsnarl", "Obstagoon", "Perrserker", "Cursola", "Sirfetch%27d", "Mr._Rime", "Runerigus", 
         "Milcery", "Alcremie", "Falinks", "Pincurchin", "Snom", "Frosmoth", "Stonjourner", "Eiscue", "Indeedee", "Morpeko", 
         "Cufant", "Copperajah", "Dracozolt", "Arctozolt", "Dracovish", "Arctovish", "Duraludon", "Dreepy", "Drakloak", "Dragapult", 
         "Zacian", "Zamazenta", "Eternatus", "Kubfu", "Urshifu", "Zarude", "Regieleki", "Regidrago", "Glastrier", "Spectrier", "Calyrex"]

**Definido a função de raspagem do site**

In [3]:
def raspagem(lista):
    entradas = {}
    for nome in lista:
        
        #obtendo o html
        string = nome
        url = 'https://bulbapedia.bulbagarden.net/wiki/'+string+'_(Pok%C3%A9mon)'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
        try:
            req = Request(url, headers = headers)
            response = urlopen(req)
            html = response.read()
        except HTTPError as e:
            print(e.status, e.reason)
        except URLError as e:
            print(e.reason)    

        #mudando para string
        html = html.decode('utf-8') 

        #removendo os espaços do html
        " ".join(html.split()).replace('> <', '><')

        #criando o objeto soup
        soup = BeautifulSoup(html, 'html.parser')

        #listando as entradas
        entries = []
        for item in soup.findAll('td', {"class" : "roundy", "style":"vertical-align: middle; border: 1px solid #9DC1B7; padding-left:3px;"}):
            entries.append(item.get_text())
        entradas[nome] = entries

    return pd.DataFrame.from_dict(entradas, orient='index')

**Execução da Raspagem**

In [4]:
df_1 = raspagem(pkmn1)
df_1

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
Bulbasaur,A strange seed was planted on its back at birt...,It can go for days without eating a single mor...,The bulb-like pouch on its back grows larger a...,The seed on its back is filled with nutrients....,It carries a seed on its back right from birth...,"While it is young, it uses the nutrients that ...",The seed on its back is filled with nutrients....,Bulbasaur can be seen napping in bright sunlig...,There is a plant seed on its back right from t...,A strange seed was planted on its back at birt...,...,None,None,None,None,None,None,None,None,None,None
Ivysaur,"When the bulb on its back grows large, it appe...",The bulb on its back grows by drawing energy. ...,The bud on its back apparently draws energy fr...,Exposure to sunlight adds to its strength. Sun...,"If the bud on its back starts to smell sweet, ...",The bulb on its back grows as it absorbs nutri...,Exposure to sunlight adds to its strength. Sun...,There is a bud on this Pokémon's back. To supp...,"To support its bulb, Ivysaur's legs grow sturd...",There is a plant bulb on its back. When it abs...,...,None,None,None,None,None,None,None,None,None,None
Venusaur,The plant blooms when it is absorbing solar en...,The flower on its back catches the sun's rays....,Absorbs solar energy as nutrition. When it is ...,By spreading the broad petals of its flower an...,It is able to convert sunlight into energy. As...,"As it warms itself and absorbs the sunlight, i...",By spreading the broad petals of its flower an...,There is a large flower on Venusaur's back. Th...,Venusaur's flower is said to take on vivid col...,A bewitching aroma wafts from its flower. The ...,...,None,None,None,None,None,None,None,None,None,None
Charmander,"Obviously prefers hot places. When it rains, s...",The flame at the tip of its tail makes a sound...,Even the newborns have flaming tails. Unfamili...,The flame on its tail shows the strength of it...,The flame on its tail indicates Charmander's l...,"If it's healthy, the flame on the tip of its t...",The flame on its tail shows the strength of it...,The flame that burns at the tip of its tail is...,The flame that burns at the tip of its tail is...,"From the time it is born, a flame burns at the...",...,None,None,None,None,None,None,None,None,None,None
Charmeleon,"When it swings its burning tail, it elevates t...",Tough fights could excite this Pokémon. When e...,It appears to be very vicious. It attacks with...,"It is very hot-headed by nature, so it constan...","It has a barbaric nature. In battle, it whips ...","If it becomes agitated during battle, it spout...","It is very hot-headed by nature, so it constan...",Charmeleon mercilessly destroys its foes using...,"Without pity, its sharp claws destroy foes. If...",It lashes about with its tail to knock down it...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Drowzee,Puts enemies to sleep then eats their dreams. ...,"If you sleep by it all the time, it will somet...",It puts its enemy to sleep and eats the victim...,"If you think that you had a good dream, but yo...",It remembers every dream it eats. It rarely ea...,"When it twitches its nose, it can tell where s...","If you think that you had a good dream, but yo...",If your nose becomes itchy while you are sleep...,If your nose becomes itchy while you are sleep...,"A descendant of the legendary animal baku, whi...",...,None,None,None,None,None,None,None,None,None,None
Hypno,"When it locks eyes with an enemy, it will use ...",Avoid eye contact if you come across one. It w...,"If you lock eyes with it, it will try to hypno...","When it is very hungry, it puts humans it meet...",Always holding a pendulum that it rocks at a s...,"The longer it swings its pendulum, the longer ...","When it is very hungry, it puts humans it meet...",Hypno holds a pendulum in its hand. The arcing...,The arcing movement an

In [5]:
df_2 = raspagem(pkmn2)
df_2

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
Electrode,It stores electric energy under very high pres...,Stores electrical energy inside its body. Even...,Stores enormous amounts of electricity within ...,It is dangerous. If it has too much electricit...,It stores an overflowing amount of electric en...,"The more energy it charges up, the faster it g...",It is dangerous. If it has too much electricit...,Electrode eats electricity in the atmosphere. ...,One of Electrode's characteristics is its attr...,They appear in great numbers at electric power...,...,None,None,None,None,None,None,None,None,None,None
Exeggcute,"Often mistaken for eggs. When disturbed, they ...",The heads attract each other and spin around. ...,A Pokémon that is like the seeds of a plant. I...,"The shell is very durable. Even if it cracks, ...","Using telepathy only they can receive, they al...","If even one is separated from the group, the e...","The shell is very durable. Even if it cracks, ...",This Pokémon consists of six eggs that form a ...,It consists of six eggs that care for each oth...,Even though it appears to be eggs of some sort...,...,These Pokémon get nervous when they're not in ...,None,None,None,None,None,None,None,None,None
Exeggutor,"Legend has it that on rare occasions, one of i...",Its cries are very noisy. This is because each...,Each of the fruits is an independently-thinkin...,"Its three heads think independently. However, ...","If a head drops off, it emits a telepathic cal...",Living in a good environment makes it grow lot...,"Its three heads think independently. However, ...",Exeggutor originally came from the tropics. It...,"Originally from the tropics, Exeggutor's heads...","It is called ""The Walking Tropical Rainforest....",...,"As it grew taller and taller, it outgrew its r...",Exeggutor is the pride of the Alolan people. I...,It swings its long neck like a whip and smacks...,The strong sunlight of the Alola region has aw...,Each of Exeggutor's three heads is thinking di...,"When they work together, Exeggutor's three hea...",Blazing sunlight has brought out the true form...,This Pokémon's psychic powers aren't as strong...,None,None
Cubone,"Because it never removes its skull helmet, no ...",Wears the skull of its deceased mother. Its cr...,Always wears the skull of its deceased mother ...,"If it is sad or lonely, the skull it wears sha...","It always wears the skull of its dead mother, ...",It lost its mother after its birth. It wears i...,"If it is sad or lonely, the skull it wears sha...",Cubone pines for the mother it will never see ...,It pines for the mother it will never see agai...,It wears the skull of its dead mother on its h...,...,This Pokémon wears the skull of its deceased m...,None,None,None,None,None,None,None,None,None
Marowak,The bone it holds is its key weapon. It throws...,"Small and weak, this Pokémon is adept with its...","Originally a small and weak Pokémon, it became...",It has been seen pounding boulders with the bo...,It collects bones from an unknown place. A Mar...,Somewhere in the world is a cemetery just for ...,It collects bones from an unknown place. A Mar...,Marowak is the evolved form of a Cubone that h...,A Marowak is the evolved form of a Cubone that...,It is small and was originally very weak. Its ...,...,Its custom is to mourn its lost companions. Mo...,The rich greenery of the Alola region is hard ...,"When it beats opponents with its bone, the cur...",It has transformed the spirit of its dear depa...,This Pokémon overcame its sorrow to evolve a s...,"When this Pokémon evolved, the skull of its mo...",This Pokémon sets the bone it holds on fire an...,The cursed flames that light up the bone carri...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Espeon,It uses the fine hair that covers its body to ...,"By reading air currents, it can predict things...",The tip of its forked tail quivers when it is ...,It uses the fine hair that c

In [6]:
df_3 = raspagem(pkmn3)
df_3

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
Unown,Their shapes look like hieroglyphs on ancient ...,"Its flat, thin body is always stuck on walls. ...","Because different types of Unown exist, it is ...",Their shapes look like hieroglyphs on ancient ...,This Pokémon is shaped like ancient writing. I...,This Pokémon is shaped like ancient text chara...,"Its flat, thin body is always stuck on walls. ...",Their shapes look like hieroglyphs on ancient ...,"Shaped like ancient writing, it is a huge myst...",They seem to communicate among each other tele...,...,This Pokémon is shaped like ancient writing. I...,None,None,None,None,None,None,None,None,None
Wobbuffet,"It hates light and shock. If attacked, it infl...","To keep its pitch-black tail hidden, it lives ...","In order to conceal its black tail, it lives i...","It hates light and shock. If attacked, it infl...","If two or more Wobbuffet meet, they will turn ...",Wobbuffet does nothing but endure attacks - it...,"Usually docile, a Wobbuffet strikes back feroc...","To keep its pitch-black tail hidden, it lives ...","It hates light and shock. If attacked, it infl...",It desperately tries to keep its black tail hi...,...,Wobbuffet does nothing but endure attacks—it w...,"It hates light and shock. If attacked, it infl...","To keep its pitch-black tail hidden, it lives ...",None,None,None,None,None,None,None
Girafarig,Its tail has a small brain of its own. Beware!...,"Its tail, which also contains a small brain, m...","When it is in danger, its tail uses some sort ...",Its tail has a small brain of its own. Beware!...,"Girafarig's rear head also has a brain, but it...",Girafarig's rear head contains a tiny brain th...,A Girafarig is an herbivore—it eats grass and ...,"Its tail, which also contains a small brain, m...",Its tail has a small brain of its own. Beware!...,Its tail also has a small brain. It bites to r...,...,Its tail has a small brain of its own. Beware!...,"Girafarig's rear head also has a brain, but it...",Girafarig's rear head contains a tiny brain th...,None,None,None,None,None,None,None
Pineco,It likes to make its shell thicker by adding l...,It hangs and waits for flying insect prey to c...,It spits out a fluid that it uses to glue tree...,It likes to make its shell thicker by adding l...,Pineco hangs from a tree branch and patiently ...,A Pineco hangs from a tree branch and waits fo...,It hangs and waits for flying insect prey to c...,It likes to make its shell thicker by adding l...,It looks just like a pinecone. Its shell prote...,It likes to make its shell thicker by adding l...,...,"Motionless, it hangs from trees, waiting for i...","It sticks tree bark to itself with its saliva,...",None,None,None,None,None,None,None,None
Forretress,Its entire body is shielded by a steel-hard sh...,It remains immovably rooted to its tree. It sc...,Usually found hanging on to a fat tree trunk. ...,Its entire body is shielded by a steel-hard sh...,Forretress conceals itself inside its hardened...,It keeps itself inside its steel shell. The sh...,It remains immovably rooted to its tree. It sc...,Its entire body is shielded by a steel-hard sh...,It is encased in a steel shell. Its peering ey...,Its entire body is shielded by a steel-hard sh...,...,"In the moment that it gulps down its prey, the...",None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Exploud,Exploud triggers earthquakes with the tremors ...,Exploud communicates its feelings to others by...,It has sound-generating organs all over its bo...,It emits a variety of sounds from the holes al...,Its howls can be heard over six miles away. It...,Its roar in battle shakes the ground like a tr...,Its howls can be heard over six miles away. It...,Its howls can be heard over six miles away. It...,Its howls can be heard over six miles away. It...,Its roar in battle shakes the ground like a tr...,...,None,None,None,None,None,None,None,No

In [7]:
df_4 = raspagem(pkmn4)
df_4

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
Skitty,Skitty has the habit of becoming fascinated by...,Skitty is known to chase around playfully afte...,A Skitty's adorably cute behavior makes it hig...,It is said to be difficult to earn its trust. ...,It can't stop itself from chasing moving thing...,It's adorable when it chases its own tail. It'...,It can't stop itself from chasing moving thing...,It shows its cute side by chasing its own tail...,It can't stop itself from chasing moving thing...,It shows its cute side by chasing its own tail...,...,None,None,None,None,None,None,None,None,None,None
Delcatty,Delcatty prefers to live an unfettered existen...,Delcatty sleeps anywhere it wants without keep...,"Rather than keeping a permanent lair, it habit...",The favorite of trend-conscious female Trainer...,It is highly popular among female Trainers for...,It dislikes dirty places. It often searches fo...,It is highly popular among female Trainers for...,The reason it does not have a nest is that it ...,It dislikes dirty places. It often searches fo...,It is highly popular among female Trainers for...,...,None,None,None,None,None,None,None,None,None,None
Sableye,Sableye lead quiet lives deep inside caverns. ...,Sableye digs the ground with sharpened claws t...,It digs branching holes in caves using its sha...,"It feeds on gemstone crystals. In darkness, it...",It hides in the darkness of caves. Its diet of...,It dwells in the darkness of caves. It uses it...,It hides in the darkness of caves. Its diet of...,It hides in the darkness of caves. Its diet of...,It hides in the darkness of caves. Its diet of...,It dwells in the darkness of caves. It uses it...,...,"Bathed in the energy of Mega Evolution, the ge...",It blocks any and all attacks with its giant-s...,This Pokémon is feared. When its gemstone eyes...,"It feeds on gemstone crystals. In darkness, it...",None,None,None,None,None,None
Mawile,Mawile's huge jaws are actually steel horns th...,Don't be taken by this Pokémon's cute face - i...,Its giant jaws are actually steel horns that t...,It uses its docile-looking face to lull foes i...,Attached to its head is a huge set of jaws for...,It chomps with its gaping mouth. Its huge jaws...,Attached to its head is a huge set of jaws for...,Attached to its head is a huge set of jaws for...,It uses its docile-looking face to lull foes i...,Attached to its head is a huge set of jaws for...,...,None,None,None,None,None,None,None,None,None,None
Aron,This Pokémon has a body of steel. To make its ...,Aron has a body of steel. With one all-out cha...,A Pokémon that is clad in steel armor. A new s...,It eats iron to build its steel body. It is a ...,"It usually lives deep in mountains. However, h...","When it evolves, it sheds the steel carapace t...","It usually lives deep in mountains. However, h...","In order to build up its steel body, it eats i...","It usually lives deep in mountains. However, h...","When it evolves, it sheds the steel carapace t...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Empoleon,The three horns that extend from its beak atte...,It swims as fast as a jet boat. The edges of i...,"If anyone were to hurt its pride, it would sla...","It avoids unnecessary disputes, but it will de...","If anyone were to hurt its pride, it would sla...","If anyone were to hurt its pride, it would sla...",It swims as fast as a jet boat. The edges of i...,The three horns that extend from its beak atte...,It swims as fast as a jet boat. The edges of i...,The three horns that extend from its beak atte...,...,None,None,None,None,None,None,None,None,None,None
Starly,"They flock in great numbers. Though small, the...","Usually with a large flock, it is barely notic...","Because they are weak individually, they form ...","They flock around mountains and fields, chasin...","Because they are weak individually, they form ...","Because they

In [8]:
df_5 = raspagem(pkmn5)
df_5

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Bibarel,It makes its nest by damming streams with bark...,"While its movements out of water are sluggish,...",A river dammed by Bibarel will never overflow ...,It busily makes its nest with stacks of branch...,A river dammed by Bibarel will never overflow ...,A river dammed by Bibarel will never overflow ...,It busily makes its nest with stacks of branch...,It makes its nest by damming streams with bark...,It busily makes its nest with stacks of branch...,It makes its nest by damming streams with bark...,...,None,None,None,None,None,None,None,None,None,None
Kricketot,"It shakes its head back to front, causing its ...",It chats with others using the sounds of its c...,"Its legs are short. Whenever it stumbles, its ...","When its antennae hit each other, it sounds li...","Its legs are short. Whenever it stumbles, its ...","Its legs are short. Whenever it stumbles, its ...",It chats with others using the sounds of its c...,"When its antennae hit each other, it sounds li...",It chats with others using the sounds of its c...,"When its antennae hit each other, it sounds li...",...,None,None,None,None,None,None,None,None,None,None
Kricketune,It crosses its knifelike arms in front of its ...,It signals its emotions with its melodies. Sci...,There is a village that hosts a contest based ...,By allowing its cry to resonate in the hollow ...,There is a village that hosts a contest based ...,There is a village that hosts a contest based ...,It crosses its knifelike arms in front of its ...,It signals its emotions with its melodies. Sci...,It crosses its knifelike arms in front of its ...,It signals its emotions with its melodies. Sci...,...,None,None,None,None,None,None,None,None,None,None
Shinx,All of its fur dazzles if danger is sensed. It...,Its forelegs have a muscle-based system of gen...,The extension and contraction of its muscles g...,The extension and contraction of its muscles g...,The extension and contraction of its muscles g...,The extension and contraction of its muscles g...,The extension and contraction of its muscles g...,All of its fur dazzles if danger is sensed. It...,The extension and contraction of its muscles g...,All of its fur dazzles if danger is sensed. It...,...,None,None,None,None,None,None,None,None,None,None
Luxio,Its claws loose electricity with enough ampera...,It rests its forelegs on others to communicate...,Strong electricity courses through the tips of...,"By gathering their tails together, they collec...",Strong electricity courses through the tips of...,Strong electricity courses through the tips of...,Its claws loose electricity with enough ampera...,Strong electricity courses through the tips of...,Its claws loose electricity with enough ampera...,Strong electricity courses through the tips of...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Snivy,It is very intelligent and calm. Being exposed...,They photosynthesize by bathing their tails in...,Being exposed to sunlight makes its movements ...,Being exposed to sunlight makes its movements ...,They photosynthesize by bathing their tails in...,Being exposed to sunlight makes its movements ...,They photosynthesize by bathing their tails in...,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Servine,It moves along the ground as if sliding. Its s...,They avoid attacks by sinking into the shadows...,"When it gets dirty, its leaves can't be used i...",It moves along the ground as if sliding. Its s...,"When it gets dirty, its leaves can't be used i...",It moves along the ground as if sliding. Its s...,"When it gets dirty, its leaves can't be used i...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
Serperior,It can stop its opponents' movements with just...,They raise their heads to intimidate opponents...,It only gives its all against strong opponents...,It only gi

In [9]:
df_6 = raspagem(pkmn6)
df_6

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Emboar,It can throw a fire punch by setting its fists...,It has mastered fast and powerful fighting mov...,A flaring beard of fire is proof that it is fi...,It can throw a fire punch by setting its fists...,It has mastered fast and powerful fighting mov...,It can throw a fire punch by setting its fists...,It has mastered fast and powerful fighting mov...,None,None,None,None,None,None,None,None
Oshawott,It fights using the scalchop on its stomach. I...,The scalchop on its stomach is made from the s...,The scalchop on its stomach isn't just used fo...,The scalchop on its stomach isn't just used fo...,It fights using the scalchop on its stomach. I...,The scalchop on its stomach isn't just used fo...,It fights using the scalchop on its stomach. I...,None,None,None,None,None,None,None,None
Dewott,Strict training is how it learns its flowing d...,Scalchop techniques differ from one Dewott to ...,"As a result of strict training, each Dewott le...",Strict training is how it learns its flowing d...,"As a result of strict training, each Dewott le...",Strict training is how it learns its flowing d...,"As a result of strict training, each Dewott le...",None,None,None,None,None,None,None,None
Samurott,One swing of the sword incorporated in its arm...,Part of the armor on its anterior legs becomes...,"In the time it takes a foe to blink, it can dr...","In the time it takes a foe to blink, it can dr...",One swing of the sword incorporated in its arm...,"In the time it takes a foe to blink, it can dr...",One swing of the sword incorporated in its arm...,None,None,None,None,None,None,None,None
Patrat,"Using food stored in cheek pouches, they can k...","Extremely cautious, they take shifts to mainta...","Extremely cautious, one of them will always be...","Using food stored in cheek pouches, they can k...","Extremely cautious, one of them will always be...","Using food stored in cheek pouches, they can k...","Extremely cautious, one of them will always be...",None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Joltik,Joltik that live in cities have learned a tech...,They attach themselves to large-bodied Pokémon...,"Since it can't generate its own electricity, i...","Since it can't generate its own electricity, i...",They attach themselves to large-bodied Pokémon...,"Since it can't generate its own electricity, i...",They attach themselves to large-bodied Pokémon...,Joltik can be found clinging to other Pokémon....,Joltik latch on to other Pokémon and suck out ...,None,None,None,None,None,None
Galvantula,"When attacked, they create an electric barrier...",They employ an electrically charged web to tra...,It creates barriers from electrified silk that...,They employ an electrically charged web to tra...,"When attacked, they create an electric barrier...",They employ an electrically charged web to tra...,"When attacked, they create an electric barrier...",It launches electrified fur from its abdomen a...,It lays traps of electrified threads near the ...,None,None,None,None,None,None
Ferroseed,"When threatened, it attacks by shooting a barr...",They stick their spikes into cave walls and ab...,It absorbs the iron it finds in the rock while...,"When threatened, it attacks by shooting a barr...",It absorbs the iron it finds in the rock while...,"When threatened, it attacks by shooting a barr...",It absorbs the iron it finds in the rock while...,"It defends itself by launching spikes, but its...",Mossy caves are their preferred dwellings. Enz...,None,None,None,None,None,None
Ferrothorn,It fights by swinging around its three spiky f...,"They attach themselves to cave ceilings, firin...",By swinging around its three spiky feels and s...,By swinging around its three spiky feelers and...,"They attach themselves to cave ceilings, firin...",By swinging around its three spiky feelers and...,"They attach themselves to cave ceilings, firin...","This Pokémon scrapes its spik

In [10]:
df_7 = raspagem(pkmn7)
df_7

,0,1,2,3,4,5,6,7,8,9,10,11,12
Klang,"By changing the direction in which it rotates,...",Spinning minigears are rotated at high speed a...,A minigear and big gear comprise its body. If ...,"By changing the direction in which it rotates,...",A minigear and big gear comprise its body. If ...,"By changing the direction in which it rotates,...",A minigear and big gear comprise its body. If ...,"When Klang goes all out, the minigear links up...",Many companies in the Galar region choose Klan...,None,None,None,None
Klinklang,Its red core functions as an energy tank. It f...,The gear with the red core is rotated at high ...,The minigear spins at high speed. Then the ene...,The gear with the red core is rotated at high ...,Its red core functions as an energy tank. It f...,The gear with the red core is rotated at high ...,Its red core functions as an energy tank. It f...,"From its spikes, it launches powerful blasts o...",The three gears that compose this Pokémon spin...,None,None,None,None
Tynamo,"While one alone doesn't have much power, a cha...",These Pokémon move in schools. They have an el...,One alone can emit only a trickle of electrici...,"While one alone doesn't have much power, a cha...",One alone can emit only a trickle of electrici...,"While one alone doesn't have much power, a cha...",One alone can emit only a trickle of electrici...,None,None,None,None,None,None
Eelektrik,They coil around foes and shock them with elec...,These Pokémon have a big appetite. When they s...,It wraps itself around its prey and paralyzes ...,It wraps itself around its prey and paralyzes ...,These Pokémon have a big appetite. When they s...,It wraps itself around its prey and paralyzes ...,These Pokémon have a big appetite. When they s...,None,None,None,None,None,None
Eelektross,They crawl out of the ocean using their arms. ...,"With their sucker mouths, they suck in prey. T...","It latches on to prey with its sucker mouth, s...","With their sucker mouths, they suck in prey. T...",They crawl out of the ocean using their arms. ...,"With their sucker mouths, they suck in prey. T...",They crawl out of the ocean using their arms. ...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Heliolisk,They flare their frills and generate energy. A...,"It stimulates its muscles with electricity, bo...",They flare their frills and generate energy. A...,"It stimulates its muscles with electricity, bo...",A now-vanished desert culture treasured these ...,One Heliolisk basking in the sun with its fril...,None,None,None,None,None,None,None
Tyrunt,This Pokémon was restored from a fossil. If so...,Its immense jaws have enough destructive force...,This Pokémon was restored from a fossil. If so...,Its immense jaws have enough destructive force...,Its large jaw has incredible destructive power...,Tyrunt is spoiled and selfish. It may just be ...,"This is an ancient Pokémon, revived in modern ...",This Pokémon is selfish and likes to be pamper...,None,None,None,None,None
Tyrantrum,"Thanks to its gargantuan jaws, which could shr...",Nothing could stop this Pokémon 100 million ye...,"Thanks to its gargantuan jaws, which could shr...",Nothing could stop this Pokémon 100 million ye...,"Complete restoration is impossible, allowing r...","The king of the ancient world, it can easily c...","This Pokémon is from about 100,000,000 years a...",A single bite of Tyrantrum's massive jaws will...,None,None,None,None,None
Amaura,This ancient Pokémon was restored from part of...,This calm Pokémon lived in a cold land where t...,This ancient Pokémon was restored from part of...,This calm Pokémon lived in a cold land where t...,It lived in cold areas in ancient times. It's ...,"Amaura was restored successfully, but it's not...",This Pokémon was successfully restored from a ...,Amaura is an ancient Pokémon that has gone ext...,None,None,None,None,None


In [11]:
df_8 = raspagem(pkmn8)
df_8

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Sylveon,It sends a soothing aura from its ribbonlike f...,It wraps its ribbonlike feelers around the arm...,It sends a soothing aura from its ribbonlike f...,It wraps its ribbonlike feelers around the arm...,Its ribbonlike feelers give off an aura that w...,"When this Pokémon sights its prey, it swirls i...","Once a fight breaks out, it will unflinchingly...",Sylveon wraps its ribbonlike feelers around it...,By releasing enmity-erasing waves from its rib...,There's a Galarian fairy tale that describes a...,None,None,None,None,None,None
Hawlucha,"Although its body is small, its proficient fig...","With its wings, it controls its position in th...","Although its body is small, its proficient fig...","With its wings, it controls its position in th...","It overwhelms opponents with quick moves, but ...","In combat, Hawlucha leaps nimbly about, taking...",It drives its opponents to exhaustion with its...,It always strikes a pose before going for its ...,None,None,None,None,None,None,None,None
Dedenne,Its whiskers serve as antennas. By sending and...,It uses its tail to absorb electricity from po...,Its whiskers serve as antennas. By sending and...,It uses its tail to absorb electricity from po...,It sneaks into people's homes and steals elect...,Its upper whiskers are sensors that survey its...,A Dedenne's whiskers pick up electrical waves ...,Since Dedenne can't generate as much electrici...,None,None,None,None,None,None,None,None
Carbink,Born from temperatures and pressures deep unde...,It has slept underground for hundreds of milli...,Born from temperatures and pressures deep unde...,It has slept underground for hundreds of milli...,Born from the high temperatures and pressures ...,"Although this Pokémon is not especially rare, ...",Some say that deep beneath the surface of the ...,To keep the jewels on their bodies free of tar...,"When beset by attackers, Carbink wipes them al...","It's said that somewhere in the world, there's...",None,None,None,None,None,None
Goomy,"The weakest Dragon-type Pokémon, it lives in d...",It's covered in a slimy membrane that makes an...,"The weakest Dragon-type Pokémon, it lives in d...",It's covered in a slimy membrane that makes an...,"The weakest of all Dragon-type Pokémon, it's u...","Its source of protection is its slimy, germ-la...",It uses its horns to check out its surrounding...,"Its body is mostly water. If it dries out, it ...","Because most of its body is water, it will dry...",Their horns are powerful sensors. As soon as G...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pheromosa,"One of the dangerous Ultra Beasts, it has been...",One of the Ultra Beasts. It refuses to touch a...,"A life-form that lives in another world, its b...",Although it's alien to this world and a danger...,None,None,None,None,None,None,None,None,None,None,None,None
Xurkitree,One of the mysterious life-forms known as Ultr...,It appeared from the Ultra Wormhole. It raided...,Although it's alien to this world and a danger...,They've been dubbed Ultra Beasts. Some of them...,None,None,None,None,None,None,None,None,None,None,None,None
Celesteela,It appeared from the Ultra Wormhole. Witnesses...,One kind of Ultra Beast. Witnesses have seen i...,"One of the dangerous UBs, high energy readings...",Although it's alien to this world and a danger...,None,None,None,None,None,None,None,None,None,None,None,None
Kartana,This Ultra Beast came from the Ultra Wormhole....,"One of the Ultra Beast life-forms, it was obse...","This Ultra Beast's body, which is as thin as p...",Although it's alien to this world and a danger...,None,None,None,None,None,None,None,None,None,None,None,None


In [12]:
df_9 = raspagem(pkmn9)
df_9

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Necrozma,"Reminiscent of the Ultra Beasts, this life-for...",Light is apparently the source of its energy. ...,It looks somehow pained as it rages around in ...,Light is the source of its energy. If it isn't...,This is its form while it is devouring the lig...,This is Necrozma's form while it's absorbing t...,Lunala no longer has a will of its own. Now un...,This is its form while it's devouring the ligh...,This is its form when it has absorbed overwhel...,The light pouring out from all over its body a...,It survives by absorbing light. After a long t...,"It needs light to survive, and it goes on a ra...","When it dominates Solgaleo, it takes on this f...",Necrozma has attached itself to Solgaleo. It s...,"Necrozma has subjugated Lunala entirely, forci...","When Necrozma latches on to Lunala, it becomes...",None,None,None,None
Magearna,"This artificial Pokémon, constructed more than...",Its mechanized body is merely a vessel. Its tr...,It synchronizes its consciousness with others ...,"Built roughly 500 years ago by a scientist, th...",This beautiful form reflects Magearna's appear...,This gorgeous Magearna looks as it did at the ...,This is its form from almost 500 years ago. It...,It functions thanks to its artificial soul. Th...,None,None,None,None,None,None,None,None,None,None,None,None
Marshadow,"Able to conceal itself in shadows, it never ap...","It lurks in the shadows of others, copying the...",It slips into the shadows of others and mimics...,It sinks into the shadows of people and Pokémo...,By slipping into the shadow of a martial arts ...,"This Pokémon can conceal itself in any shadow,...",None,None,None,None,None,None,None,None,None,None,None,None,None,None
Poipole,This Ultra Beast is well enough liked to be ch...,An Ultra Beast that lives in a different world...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Naganadel,It stores hundreds of liters of poisonous liqu...,"One kind of Ultra Beast, it fires a glowing, v...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Regieleki,This Pokémon is a cluster of electrical energy...,Its entire body is made up of a single organ t...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Regidrago,An academic theory proposes that Regidrago's a...,Its body is composed of crystallized dragon en...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Glastrier,Glastrier emits intense cold from its hooves. ...,"Glastrier has tremendous physical strength, an...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Spectrier,It probes its surroundings with all its senses...,"As it dashes through the night, Spectrier abso...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


**Concatenando os dataframes**

In [13]:
df_pkmn = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9], ignore_index=False)
df_pkmn

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
Bulbasaur,A strange seed was planted on its back at birt...,It can go for days without eating a single mor...,The bulb-like pouch on its back grows larger a...,The seed on its back is filled with nutrients....,It carries a seed on its back right from birth...,"While it is young, it uses the nutrients that ...",The seed on its back is filled with nutrients....,Bulbasaur can be seen napping in bright sunlig...,There is a plant seed on its back right from t...,A strange seed was planted on its back at birt...,...,None,None,None,None,None,None,None,None,None,None
Ivysaur,"When the bulb on its back grows large, it appe...",The bulb on its back grows by drawing energy. ...,The bud on its back apparently draws energy fr...,Exposure to sunlight adds to its strength. Sun...,"If the bud on its back starts to smell sweet, ...",The bulb on its back grows as it absorbs nutri...,Exposure to sunlight adds to its strength. Sun...,There is a bud on this Pokémon's back. To supp...,"To support its bulb, Ivysaur's legs grow sturd...",There is a plant bulb on its back. When it abs...,...,None,None,None,None,None,None,None,None,None,None
Venusaur,The plant blooms when it is absorbing solar en...,The flower on its back catches the sun's rays....,Absorbs solar energy as nutrition. When it is ...,By spreading the broad petals of its flower an...,It is able to convert sunlight into energy. As...,"As it warms itself and absorbs the sunlight, i...",By spreading the broad petals of its flower an...,There is a large flower on Venusaur's back. Th...,Venusaur's flower is said to take on vivid col...,A bewitching aroma wafts from its flower. The ...,...,None,None,None,None,None,None,None,None,None,None
Charmander,"Obviously prefers hot places. When it rains, s...",The flame at the tip of its tail makes a sound...,Even the newborns have flaming tails. Unfamili...,The flame on its tail shows the strength of it...,The flame on its tail indicates Charmander's l...,"If it's healthy, the flame on the tip of its t...",The flame on its tail shows the strength of it...,The flame that burns at the tip of its tail is...,The flame that burns at the tip of its tail is...,"From the time it is born, a flame burns at the...",...,None,None,None,None,None,None,None,None,None,None
Charmeleon,"When it swings its burning tail, it elevates t...",Tough fights could excite this Pokémon. When e...,It appears to be very vicious. It attacks with...,"It is very hot-headed by nature, so it constan...","It has a barbaric nature. In battle, it whips ...","If it becomes agitated during battle, it spout...","It is very hot-headed by nature, so it constan...",Charmeleon mercilessly destroys its foes using...,"Without pity, its sharp claws destroy foes. If...",It lashes about with its tail to knock down it...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Regieleki,This Pokémon is a cluster of electrical energy...,Its entire body is made up of a single organ t...,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Regidrago,An academic theory proposes that Regidrago's a...,Its body is composed of crystallized dragon en...,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glastrier,Glastrier emits intense cold from its hooves. ...,"Glastrier has tremendous physical strength, an...",None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Spectrier,It probes its surroundings with all its senses...,"As it dashes through the night, Spectrier abso...",None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Exportando para CSV**

In [14]:
compression_opts = dict(method='zip', archive_name='bulbapedia.csv')  
df_pkmn.to_csv('bulbapedia.zip', index=True, compression=compression_opts)